# Terraform Resource


## Running locally

1. Docker Compose file:

```yaml
services:
  localstack:
    image: localstack/localstack:latest
    ports:
      - "4566:4566"
    environment:
      - SERVICES=iam,s3,lambda,dynamodb,sqs,cloudwatch,logs
      - DEBUG=1
      - DOCKER_HOST=unix:///var/run/docker.sock
      - AWS_DEFAULT_REGION=us-east-1
      - LOCALSTACK_HOST=localstack
    volumes:
      - /var/run/docker.sock:/var/run/docker.sock
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:4566/health"]
      interval: 10s
      timeout: 5s
      retries: 3
```

2. Run image:

```sh
docker compose run --rm localstack
```


Install packages


In [3]:
!uv pip install -q \
    boto3==1.42.16

import packages


In [ ]:
import boto3
from botocore.exceptions import ClientError

Define terraform file:

```hcl title='main.tf'
--8<-- "docs/notebooks/python/iac/terraform-resource/main.tf"
```


Initialize terraform


In [15]:
!terraform init

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v6.27.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


Plan to add resources


In [14]:
!terraform plan


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_s3_bucket.some_bucket will be created
  + resource "aws_s3_bucket" "some_bucket" {
      + acceleration_status         = (known after apply)
      + acl                         = (known after apply)
      + arn                         = (known after apply)
      + bucket                      = "some-bucket"
      + bucket_domain_name          = (known after apply)
      + bucket_prefix               = (known after apply)
      + bucket_region               = (known after apply)
      + bucket_regional_domain_name = (known after apply)
      + force_destroy               = false
      + hosted_zone_id              = (known after apply)
      + id                          = (known after apply)
      + object_lock_enabled         = (known after apply)
      + policy                    

Apply plan


In [17]:
!terraform apply --auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_s3_bucket.some_bucket will be created
  + resource "aws_s3_bucket" "some_bucket" {
      + acceleration_status         = (known after apply)
      + acl                         = (known after apply)
      + arn                         = (known after apply)
      + bucket                      = "some-bucket"
      + bucket_domain_name          = (known after apply)
      + bucket_prefix               = (known after apply)
      + bucket_region               = (known after apply)
      + bucket_regional_domain_name = (known after apply)
      + force_destroy               = false
      + hosted_zone_id              = (known after apply)
      + id                          = (known after apply)
      + object_lock_enabled         = (known after apply)
      + policy                    

Define s3 client


In [ ]:
s3_client = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id="test",
    aws_secret_access_key="test",
    endpoint_url="http://localhost:4566",
    config=boto3.session.Config(s3={"addressing_style": "path"}),
)

bucket_name = "some-bucket"

Check if bucket exists


In [ ]:
try:
    s3_client.head_bucket(Bucket=bucket_name)
    print("Bucket exists")
except ClientError as error:
    if error.response["Error"]["Code"] == "404":
        print("Bucket does not exist, creating it")
        s3_client.create_bucket(Bucket=bucket_name)
    else:
        raise

Bucket exists


In [24]:
!terraform destroy --auto-approve

aws_s3_bucket.some_bucket: Refreshing state... [id=some-bucket]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_s3_bucket.some_bucket will be destroyed
  - resource "aws_s3_bucket" "some_bucket" {
      - arn                         = "arn:aws:s3:::some-bucket" -> null
      - bucket                      = "some-bucket" -> null
      - bucket_domain_name          = "some-bucket.s3.amazonaws.com" -> null
      - bucket_region               = "us-east-1" -> null
      - bucket_regional_domain_name = "some-bucket.s3.us-east-1.amazonaws.com" -> null
      - force_destroy               = false -> null
      - hosted_zone_id              = "Z3AQBSTGFYJSTF" -> null
      - id                          = "some-bucket" -> null
      - object_lock_enabled         = false -> null
      - region                      = "us-east-1" -> null
   